In [ ]:
# PRACTICAL Odoo 16 Invoice Import


!pip install PyPDF2 pdfplumber openpyxl pandas

import pandas as pd
import json
import re
import os
from google.colab import files
import pdfplumber
import base64
from datetime import datetime

print("🔧 PRACTICAL Odoo 16 Invoice Import Solutions")
print("=" * 60)

def parse_indonesian_number(num_str):
    """Convert Indonesian formatted number to float"""
    if not num_str or str(num_str).strip() == '':
        return 0.0

    clean_str = str(num_str).replace('Rp', '').replace('IDR', '').strip()

    try:

        if ',' in clean_str and '.' in clean_str:
            parts = clean_str.rsplit(',', 1)
            integer_part = parts[0].replace('.', '')
            decimal_part = parts[1]
            return float(f"{integer_part}.{decimal_part}")
        elif '.' in clean_str and clean_str.count('.') > 1:
            # Multiple dots = thousands separators
            return float(clean_str.replace('.', ''))
        elif ',' in clean_str and clean_str.count(',') == 1:
            # Single comma = decimal separator
            return float(clean_str.replace(',', '.'))
        else:
            return float(clean_str.replace(',', ''))
    except:
        print(f"Warning: Could not parse number '{num_str}'")
        return 0.0

def extract_pdf_text(pdf_path):
    """Extract text from PDF"""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text
    except Exception as e:
        print(f"Error extracting PDF text: {e}")
        return None

def parse_invoice_comprehensive(pdf_text):
    """Enhanced invoice parsing for Indonesian invoices"""
    data = {
        'invoice_number': '',
        'invoice_date': '',
        'due_date': '',
        'supplier_name': '',
        'supplier_address': '',
        'supplier_npwp': '',
        'supplier_email': '',
        'customer_name': '',
        'customer_address': '',
        'customer_npwp': '',
        'line_items': [],
        'subtotal': 0.0,
        'tax_amount': 0.0,
        'total_amount': 0.0,
        'currency': 'IDR'
    }

    lines = [line.strip() for line in pdf_text.split('\n') if line.strip()]

    # Extract invoice number - multiple patterns
    for line in lines:
        patterns = [
            r'INV[/\-\w]*[/\-]\d{4}[/\-]\d+',
            r'INVOICE\s*NO[:\s]*([A-Z0-9/\-]+)',
            r'No\.\s*Invoice[:\s]*([A-Z0-9/\-]+)',
            r'Invoice\s*#[:\s]*([A-Z0-9/\-]+)'
        ]
        for pattern in patterns:
            match = re.search(pattern, line, re.IGNORECASE)
            if match:
                if len(pattern.split('(')) > 1:  # Has capture group
                    data['invoice_number'] = match.group(1).strip()
                else:
                    data['invoice_number'] = match.group().strip()
                break
        if data['invoice_number']:
            break

    # Extract dates
    date_patterns = [r'\d{1,2}/\d{1,2}/\d{4}', r'\d{1,2}-\d{1,2}-\d{4}']
    for line in lines:
        for date_pattern in date_patterns:
            dates_in_line = re.findall(date_pattern, line)
            if dates_in_line:
                if 'date' in line.lower() and 'due' not in line.lower():
                    data['invoice_date'] = dates_in_line[0]
                elif 'due' in line.lower():
                    data['due_date'] = dates_in_line[0]

    # Extract supplier info
    supplier_keywords = ['PT.', 'CV.', 'UD.', 'From:', 'Vendor:']
    for i, line in enumerate(lines):
        for keyword in supplier_keywords:
            if keyword in line and not data['supplier_name']:
                if keyword in ['From:', 'Vendor:']:
                    # Next line is company name
                    if i + 1 < len(lines):
                        data['supplier_name'] = lines[i + 1]
                else:
                    # Current line contains company name
                    data['supplier_name'] = line

                # Look for NPWP in next few lines
                for j in range(i, min(i + 8, len(lines))):
                    if 'npwp' in lines[j].lower():
                        npwp_match = re.search(r'[\d\.\-]{10,}', lines[j])
                        if npwp_match:
                            data['supplier_npwp'] = npwp_match.group()

                    # Look for email
                    if '@' in lines[j]:
                        email_match = re.search(r'[\w\.-]+@[\w\.-]+\.\w+', lines[j])
                        if email_match:
                            data['supplier_email'] = email_match.group()
                break
        if data['supplier_name']:
            break

    # Extract customer info
    customer_keywords = ['Bill To:', 'Invoice To:', 'Customer:', 'To:']
    for i, line in enumerate(lines):
        for keyword in customer_keywords:
            if keyword in line and not data['customer_name']:
                if i + 1 < len(lines):
                    data['customer_name'] = lines[i + 1]

                # Look for customer NPWP
                for j in range(i + 1, min(i + 10, len(lines))):
                    if j < len(lines) and 'npwp' in lines[j].lower():
                        npwp_match = re.search(r'[\d\.\-]{10,}', lines[j])
                        if npwp_match:
                            data['customer_npwp'] = npwp_match.group()
                        break
                break
        if data['customer_name']:
            break

    # Extract line items - look for common patterns
    item_patterns = [
        r'(.*?)\s+(\d+(?:[,\.]\d+)?)\s+Rp\s*([\d,\.]+)\s+Rp\s*([\d,\.]+)',
        r'(.*?)\s+Rp\s*([\d,\.]+)\s+Rp\s*([\d,\.]+)',
        r'(Monthly|Service|Additional|FB-.*?)\s+.*?Rp\s*([\d,\.]+)'
    ]

    for line in lines:
        for pattern in item_patterns:
            match = re.search(pattern, line, re.IGNORECASE)
            if match:
                groups = match.groups()
                if len(groups) >= 2:
                    description = groups[0].strip()
                    # Get the last amount as total
                    amount_str = groups[-1]
                    amount = parse_indonesian_number(amount_str)

                    if description and amount > 0:
                        data['line_items'].append({
                            'description': description,
                            'amount': amount,
                            'quantity': 1
                        })
                break

    # Extract totals
    total_keywords = {
        'subtotal': ['subtotal', 'sub total', 'sub-total'],
        'tax_amount': ['tax', 'ppn', 'vat', 'pajak'],
        'total_amount': ['total', 'grand total', 'amount due', 'balance due']
    }

    for line in lines:
        line_lower = line.lower()

        # Find amounts in the line
        amounts = re.findall(r'rp\s*([\d,\.]+)', line, re.IGNORECASE)
        if not amounts:
            amounts = re.findall(r'([\d,\.]+)', line)

        if amounts:
            amount = parse_indonesian_number(amounts[-1])  # Take last amount

            for field, keywords in total_keywords.items():
                for keyword in keywords:
                    if keyword in line_lower and amount > 0:
                        if data[field] == 0.0:  # Only set if not already set
                            data[field] = amount
                        break

    return data

def create_odoo_csv_import(invoice_data, filename):
    """Create CSV file for Odoo import - MOST RELIABLE METHOD"""
    csv_filename = filename.replace('.pdf', '_vendor_bills.csv')

    # Prepare vendor bill data
    if invoice_data['line_items']:
        # Create one row per line item
        csv_data = []
        for item in invoice_data['line_items']:
            amount_excl_tax = item['amount'] / 1.11 if invoice_data['tax_amount'] > 0 else item['amount']
            csv_data.append({
                'partner_id/name': invoice_data['supplier_name'],
                'partner_id/vat': invoice_data['supplier_npwp'],
                'partner_id/email': invoice_data['supplier_email'],
                'ref': invoice_data['invoice_number'],
                'invoice_date': invoice_data['invoice_date'],
                'invoice_date_due': invoice_data['due_date'],
                'move_type': 'in_invoice',
                'currency_id': 'IDR',
                'invoice_line_ids/name': item['description'],
                'invoice_line_ids/quantity': item['quantity'],
                'invoice_line_ids/price_unit': amount_excl_tax,
                'invoice_line_ids/account_id': 'Expenses',  # You may need to adjust
                'invoice_line_ids/tax_ids/name': 'PPN 11%' if invoice_data['tax_amount'] > 0 else ''
            })
    else:
        # Single line invoice
        amount_excl_tax = invoice_data['subtotal'] if invoice_data['subtotal'] > 0 else invoice_data['total_amount'] / 1.11
        csv_data = [{
            'partner_id/name': invoice_data['supplier_name'],
            'partner_id/vat': invoice_data['supplier_npwp'],
            'partner_id/email': invoice_data['supplier_email'],
            'ref': invoice_data['invoice_number'],
            'invoice_date': invoice_data['invoice_date'],
            'invoice_date_due': invoice_data['due_date'],
            'move_type': 'in_invoice',
            'currency_id': 'IDR',
            'invoice_line_ids/name': f'Invoice {invoice_data["invoice_number"]}',
            'invoice_line_ids/quantity': 1,
            'invoice_line_ids/price_unit': amount_excl_tax,
            'invoice_line_ids/account_id': 'Expenses',
            'invoice_line_ids/tax_ids/name': 'PPN 11%' if invoice_data['tax_amount'] > 0 else ''
        }]

    df = pd.DataFrame(csv_data)
    df.to_csv(csv_filename, index=False)
    return csv_filename

def create_manual_entry_guide(invoice_data, filename):
    """Create step-by-step manual entry guide"""
    guide_filename = filename.replace('.pdf', '_manual_entry_guide.txt')

    guide_content = f"""
ODOO 16 MANUAL ENTRY GUIDE
{invoice_data['invoice_number']}
=====================================

STEP 1: Create/Update Vendor
----------------------------
Go to: Accounting → Vendors → Vendors → Create

Vendor Name: {invoice_data['supplier_name']}
Tax ID (NPWP): {invoice_data['supplier_npwp']}
Email: {invoice_data['supplier_email']}
Address: {invoice_data['supplier_address']}

STEP 2: Create Vendor Bill
-------------------------
Go to: Accounting → Vendors → Bills → Create

Basic Information:
• Vendor: {invoice_data['supplier_name']}
• Bill Reference: {invoice_data['invoice_number']}
• Bill Date: {invoice_data['invoice_date']}
• Due Date: {invoice_data['due_date']}
• Currency: IDR

STEP 3: Add Invoice Lines
------------------------
"""

    if invoice_data['line_items']:
        for i, item in enumerate(invoice_data['line_items'], 1):
            guide_content += f"""
Line {i}:
• Label: {item['description']}
• Account: Expenses (or appropriate expense account)
• Quantity: {item['quantity']}
• Unit Price: {item['amount'] / 1.11:.2f} (excluding tax)
• Taxes: PPN 11% (if applicable)
"""
    else:
        guide_content += f"""
Single Line:
• Label: Invoice {invoice_data['invoice_number']}
• Account: Expenses
• Quantity: 1
• Unit Price: {invoice_data['total_amount'] / 1.11:.2f} (excluding tax)
• Taxes: PPN 11%
"""

    guide_content += f"""

STEP 4: Verify Totals
--------------------
Expected totals:
• Untaxed Amount: {invoice_data['subtotal']:.2f}
• Tax Amount: {invoice_data['tax_amount']:.2f}
• Total: {invoice_data['total_amount']:.2f}

STEP 5: Save and Confirm
-----------------------
• Click Save
• Review the bill
• Click Confirm to post the bill

TROUBLESHOOTING:
• If vendor doesn't exist: Create it first
• If accounts missing: Check Chart of Accounts
• If taxes wrong: Configure Indonesian tax rates
• If currency issues: Enable IDR currency
"""

    with open(guide_filename, 'w', encoding='utf-8') as f:
        f.write(guide_content)

    return guide_filename

def create_excel_template(invoice_data, filename):
    """Create Excel template for bulk import"""
    excel_filename = filename.replace('.pdf', '_excel_template.xlsx')

    # Create DataFrame with proper Odoo import structure
    if invoice_data['line_items']:
        excel_data = []
        for item in invoice_data['line_items']:
            amount_excl_tax = item['amount'] / 1.11 if invoice_data['tax_amount'] > 0 else item['amount']
            excel_data.append({
                'Vendor': invoice_data['supplier_name'],
                'Bill Reference': invoice_data['invoice_number'],
                'Bill Date': invoice_data['invoice_date'],
                'Due Date': invoice_data['due_date'],
                'Description': item['description'],
                'Quantity': item['quantity'],
                'Unit Price (Excl Tax)': amount_excl_tax,
                'Tax': 'PPN 11%' if invoice_data['tax_amount'] > 0 else '',
                'Total Amount': item['amount']
            })
    else:
        amount_excl_tax = invoice_data['total_amount'] / 1.11
        excel_data = [{
            'Vendor': invoice_data['supplier_name'],
            'Bill Reference': invoice_data['invoice_number'],
            'Bill Date': invoice_data['invoice_date'],
            'Due Date': invoice_data['due_date'],
            'Description': f'Invoice {invoice_data["invoice_number"]}',
            'Quantity': 1,
            'Unit Price (Excl Tax)': amount_excl_tax,
            'Tax': 'PPN 11%' if invoice_data['tax_amount'] > 0 else '',
            'Total Amount': invoice_data['total_amount']
        }]

    df = pd.DataFrame(excel_data)
    df.to_excel(excel_filename, index=False, sheet_name='Vendor Bills')
    return excel_filename

# Main execution
print("\n Upload your PDF invoice:")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\n Processing: {filename}")

    # Extract and parse
    pdf_text = extract_pdf_text(filename)
    if not pdf_text:
        print("❌ Could not extract text from PDF")
        continue

    print(f"📄 Extracted {len(pdf_text)} characters of text")

    # Parse invoice data
    invoice_data = parse_invoice_comprehensive(pdf_text)

    # Display extracted data
    print(f"\n📋 EXTRACTED DATA:")
    print(f"📄 Invoice: {invoice_data['invoice_number']}")
    print(f"📅 Date: {invoice_data['invoice_date']}")
    print(f"🏢 Supplier: {invoice_data['supplier_name']}")
    print(f"🆔 Supplier NPWP: {invoice_data['supplier_npwp']}")
    print(f"👤 Customer: {invoice_data['customer_name']}")
    print(f"💰 Total: IDR {invoice_data['total_amount']:,.2f}")
    print(f"🧾 Line Items: {len(invoice_data['line_items'])}")

    if invoice_data['line_items']:
        print("\n📋 Line Items:")
        for i, item in enumerate(invoice_data['line_items'], 1):
            print(f"   {i}. {item['description']}: IDR {item['amount']:,.2f}")

    # Create all import formats
    files_created = []

    try:
        # 1. CSV Import (Most reliable)
        csv_file = create_odoo_csv_import(invoice_data, filename)
        files_created.append(csv_file)
        print(f"✅ Created: {csv_file}")
    except Exception as e:
        print(f"❌ CSV creation failed: {e}")

    try:
        # 2. Manual entry guide
        guide_file = create_manual_entry_guide(invoice_data, filename)
        files_created.append(guide_file)
        print(f"✅ Created: {guide_file}")
    except Exception as e:
        print(f"❌ Guide creation failed: {e}")

    try:
        # 3. Excel template
        excel_file = create_excel_template(invoice_data, filename)
        files_created.append(excel_file)
        print(f"✅ Created: {excel_file}")
    except Exception as e:
        print(f"❌ Excel creation failed: {e}")

    # 4. JSON backup
    json_filename = filename.replace('.pdf', '_extracted_data.json')
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(invoice_data, f, indent=2, ensure_ascii=False)
    files_created.append(json_filename)

    # Create download interface
    print(f"\n📦 Created {len(files_created)} files for Odoo import")

    from IPython.display import HTML, display

    # Create comprehensive download interface
    html_content = f"""
    <div style="margin: 20px 0; padding: 20px; background: #f8f9fa; border-radius: 10px;">
        <h3 style="color: #28a745;">🎯 Odoo 16 Import Solutions</h3>

        <div style="background: #d4edda; padding: 15px; border-radius: 5px; margin: 15px 0;">
            <h4 style="color: #155724;">🥇 METHOD 1: CSV Import (Recommended)</h4>
    """

    # Add download buttons for each file
    try:
        for file_path in files_created:
            with open(file_path, 'rb') as f:
                file_data = base64.b64encode(f.read()).decode()

            if file_path.endswith('.csv'):
                color = "#28a745"
                icon = "📊"
                method = "CSV Import"
                instruction = "Accounting → Vendor Bills → Import → Upload CSV"
            elif file_path.endswith('.txt'):
                color = "#17a2b8"
                icon = "📖"
                method = "Manual Entry Guide"
                instruction = "Follow step-by-step instructions"
            elif file_path.endswith('.xlsx'):
                color = "#007bff"
                icon = "📑"
                method = "Excel Template"
                instruction = "Edit and import via Excel"
            else:
                color = "#6c757d"
                icon = "💾"
                method = "Data Backup"
                instruction = "JSON data for reference"

            mime_type = {
                '.csv': 'text/csv',
                '.txt': 'text/plain',
                '.xlsx': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                '.json': 'application/json'
            }.get(os.path.splitext(file_path)[1], 'application/octet-stream')

            html_content += f"""
            <p style="margin: 10px 0;">
                <a href="data:{mime_type};base64,{file_data}"
                   download="{os.path.basename(file_path)}"
                   style="background: {color}; color: white; padding: 10px 20px;
                          text-decoration: none; border-radius: 5px; margin-right: 10px; font-weight: bold;">
                    {icon} Download {os.path.basename(file_path)}
                </a>
                <span style="color: #495057;">{method}</span><br>
                <small style="color: #6c757d; margin-left: 10px;">{instruction}</small>
            </p>
            """

        html_content += f"""
        </div>

        <div style="background: #fff3cd; padding: 15px; border-radius: 5px; margin: 15px 0;">
            <h4 style="color: #856404;">⚠️ Important Notes</h4>
            <ul style="color: #856404;">
                <li><strong>Odoo's PDF auto-import is a PAID service</strong> - requires credits/subscription</li>
                <li><strong>CSV import works 100%</strong> - no OCR or AI needed</li>
                <li><strong>Manual entry is always reliable</strong> - use the guide for step-by-step process</li>
                <li><strong>Check vendor exists first</strong> - create vendor before importing bills</li>
                <li><strong>Verify account mapping</strong> - adjust expense accounts as needed</li>
            </ul>
        </div>

        <div style="background: #d1ecf1; padding: 15px; border-radius: 5px;">
            <h4 style="color: #0c5460;">🛠️ Odoo Configuration Needed</h4>
            <ol style="color: #0c5460;">
                <li>Enable IDR currency: Accounting → Configuration → Currencies</li>
                <li>Setup Indonesian taxes: Accounting → Configuration → Taxes</li>
                <li>Create PPN 11% tax rate</li>
                <li>Verify chart of accounts has expense accounts</li>
            </ol>
        </div>
    </div>
    """

        display(HTML(html_content))

    except Exception as e:
        print(f"Could not create download interface: {e}")
        print(f"Files created: {files_created}")

        # Manual download commands
        print(f"\n🔧 Manual download commands:")
        for file_path in files_created:
            print(f"files.download('{file_path}')")

print(f"\n✅ Processing complete!")
print(f"💡 Key insight: Odoo's PDF auto-import is a PAID cloud service")
print(f"🎯 Use CSV import for 100% reliable results without extra costs")